<a href="https://colab.research.google.com/github/woojin-blaze/E-commerce-llm/blob/main/blaze_finetune_solar_ko_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64

In [ ]:
!pip install --upgrade datasets

In [ ]:
# Model from Hugging Face hub
#base_model = "NousResearch/Llama-2-7b-chat-hf"
base_model = "beomi/OPEN-SOLAR-KO-10.7B"

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
#!pip install bitsandbytes==0.39.0

In [ ]:

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
#!pip install accelerate==0.21.0

In [ ]:
#!pip install -U accelerate

In [ ]:
#!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/ColabNotebooks"

Mounted at /content/drive
blaze_finetune_llama2_base.ipynb  finetune_llama2_base_data_테스트.ipynb  results
data				  generated_text			  solarko_finetuning.ipynb
data3.csv			  nyk-cid-solarko-finetuning.ipynb
eval				  nyk_data_evaluation.ipynb


In [ ]:
#data = load_dataset("csv", data_files="/content/drive/MyDrive/ColabNotebooks/data/toy_0618.csv")
#val_data = load_dataset("csv", data_files="/content/drive/MyDrive/ColabNotebooks/data/toy_0618.csv")
data = load_dataset("csv", data_files="/content/drive/MyDrive/ColabNotebooks/data/base0618_train.csv")
val_data =load_dataset("csv", data_files="/content/drive/MyDrive/ColabNotebooks/data/base0618_val.csv")
data

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3975
    })
})

In [ ]:
data['train'][0]

{'text': '### Instruction: Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be relevant to the raw source.\n\n### Raw Source:2023 신형 스마트 터치 LED 책상용 램프 눈보호 램프 학생 기숙사용 침대 옆 조명기구 충전 가능 삼단 조명 조절\n### Detailed Description:2023년 신형 스마트 터치 LED 책상용 램프는 눈의 피로를 줄이는 눈보호 기능을 자랑합니다. \n삼단 조명 조절 기능이 탑재되어 있어, 학습이나 독서, 휴식 시에 적합한 광도를 선택할 수 있습니다.\n터치 한 번으로 조명의 밝기와 모드를 쉽게 조절할 수 있는 사용자 친화적인 디자인입니다.\n충전 가능한 기능이 있어, 기숙사나 침대 옆에서도 편리하게 사용할 수 있습니다.\n현대적인 디자인과 기능성을 겸비하여, 학생들의 책상 위에 최적의 조명 솔루션을 제공합니다.'}

In [ ]:
data['train'][1]

{'text': '### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.\n\n### Raw Source:2023 신형 스마트 로봇 청소기 가정용 전자동 물걸레 청소기 스마트 내비게이션 원격 제어로 편리하고 효율적입니다\n### Product Title:2023 스마트 로봇 청소기 자동 물걸레 내비게이션 원격제어'}

In [ ]:
val_data['train'][0]

{'text': '### Instruction: Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable.\n\n### Raw Source:레트로 순은 펜던트 목걸이 여성 패션 액세서리 보헤미안 스타일 긴 체인 크리스탈 하트 장식 발렌타인데이 선물\n### Related Keywords:[보헤미안 목걸이, 순은 펜던트, 여성 패션 액세서리, 크리스탈 하트 목걸이, 레트로 목걸이, 발렌타인데이 선물, 긴 체인 목걸이]'}

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device ="cpu"
device

'cuda'

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    #device_map = "auto"
    device_map={"": 0}
    #device_map="cpu"
)
#device = torch.device('cpu')
#model = model.to(device)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/87.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
'''
def tokenize_function(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=2048)
    outputs = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=2048)
    inputs['labels'] = outputs['input_ids']
    return inputs'''

'\ndef tokenize_function(examples):\n    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=2048)\n    outputs = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=2048)\n    inputs[\'labels\'] = outputs[\'input_ids\']\n    return inputs'

In [ ]:
'''
def formatting_prompts_func(example):

    input_text = example['input']

    instruction_start = input_text.find('[INST]') + len('[INST]')
    instruction_end = input_text.find('[/INST]')
    instruction = input_text[instruction_start:instruction_end].strip()
    raw_source = input_text[instruction_end + len('[/INST]'):].strip()

    text = f"### Instruction: {instruction}\n### Raw Source: {raw_source}\n### Response: {example['output']}"
    return {"text": text}
  '''

'\ndef formatting_prompts_func(example):\n\n    input_text = example[\'input\']\n\n    instruction_start = input_text.find(\'[INST]\') + len(\'[INST]\')\n    instruction_end = input_text.find(\'[/INST]\')\n    instruction = input_text[instruction_start:instruction_end].strip()\n    raw_source = input_text[instruction_end + len(\'[/INST]\'):].strip()\n\n    text = f"### Instruction: {instruction}\n### Raw Source: {raw_source}\n### Response: {example[\'output\']}"\n    return {"text": text}\n  '

In [ ]:
#tokenized_train_dataset = data['train'].map(tokenize_function, batched=True)
#tokenized_val_dataset = val_data['train'].map(tokenize_function, batched=True)

In [ ]:
#tokenized_train_dataset

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    #no_cuda=True #train in CPU
)
'''
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
)'''

'\ntraining_params = TrainingArguments(\n    output_dir="./results",\n    num_train_epochs=1,\n    per_device_train_batch_size=4,\n    optim="paged_adamw_32bit",\n    save_steps=25,\n    logging_steps=25,\n    learning_rate=2e-4,\n    weight_decay=0.001,\n    fp16=False,\n)'

In [ ]:
import gc
gc.collect()

99

In [ ]:
instruction_template = "### Instruction:"
response_template = "### Response:"

In [ ]:
'''
collator = DataCollatorForCompletionOnlyLM(
    instruction_template=instruction_template,
    response_template=response_template,
    tokenizer=tokenizer,
    mlm=False
)'''

'\ncollator = DataCollatorForCompletionOnlyLM(\n    instruction_template=instruction_template,\n    response_template=response_template,\n    tokenizer=tokenizer,\n    mlm=False\n)'

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    eval_dataset=val_data['train'],
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_params,
    #data_collator=collator,
    packing=False,
    #formatting_func=formatting_prompts_func
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the

Map:   0%|          | 0/3975 [00:00<?, ? examples/s]

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,1.559100
50,1.409800
75,1.128300
100,1.089000
125,1.010300
150,1.073800
175,1.007200
200,0.955300
225,0.960300
250,0.975800


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=2982, training_loss=0.697702768423028, metrics={'train_runtime': 5319.39, 'train_samples_per_second': 2.242, 'train_steps_per_second': 0.561, 'total_flos': 1.0429465437551002e+17, 'train_loss': 0.697702768423028, 'epoch': 3.0})

In [ ]:
new_model = "/content/drive/MyDrive/ColabNotebooks/results/solar-ko-3epoch"

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/ColabNotebooks/results/solar-ko-3epoch/tokenizer_config.json',
 '/content/drive/MyDrive/ColabNotebooks/results/solar-ko-3epoch/special_tokens_map.json',
 '/content/drive/MyDrive/ColabNotebooks/results/solar-ko-3epoch/tokenizer.json')

In [ ]:
'''
import matplotlib.pyplot as plt
plt.plot(lr_logger.learning_rates)
plt.xlabel('Step')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule')
plt.show()
'''


"\nimport matplotlib.pyplot as plt\nplt.plot(lr_logger.learning_rates)\nplt.xlabel('Step')\nplt.ylabel('Learning Rate')\nplt.title('Learning Rate Schedule')\nplt.show()\n"

Generated Title: ### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.
### Raw Source: 새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌
### Response: 2023 스마트워치 운동 심박수 혈압 수면 추적 남녀공용 블루투스 팔찌)
### Explanation: 이 제품은 터치 스크린을 통해 사용자의 운동 활동을 정밀하게 모니터링하며, 심박수와 혈압 등 건강 상태를 추적합니다.
남녀 공용 디자인으로 세련되면서도 실용적인 패션을 완성시켜 줍니다.
블루투스 기능이 탑재되어 있어 스마트폰과 연동하여 통화와 음악 재생이 가능합니다.
다양한 기능을 갖추고 있어 일상생활에서의 편리함을 제공합니다.
### Related Search: 스마트워치 운동 추적, 심박수 측정 스마트워치, 혈압 측정 스마트워치, 수면 추적 스마트워치, 블루투스 스마트워치, 남녀공용 스마트워치
### Keyword: 스마트워치 2023, 스마트워치 운동 추적, 스마트워치 혈압 측정, 스마트워치 수면 추적, 블루투스 스마트워치, 남녀공용 스마트워치
### Product Title: 2023 스마트워치 운동 심박수 혈압 수면 추적 남녀공용 블루투스 팔찌)
### Product Description: 이 스마트워치는 다양한 건강 모니터링 기능을 탑재하여 일상의 건강 관리를 쉽게 도와줍니다.
운동 및 일상 활동의 추적을 통해 사용자의 운동량과 카피틴을 정확히 측정할 수 있습니다.
심박수와 �


### Instruction: Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be relevant to the raw source.
### Raw Source:전신 2024형 스마트 스포츠 워치 남녀공용 심박수 모니터링 블루투스 전화 팔찌 GPS 위치 추적 방수 다기능 스포츠 모드
### Response:2024형 전신 스마트 스포츠 워치는 남녀 모두에게 적합하며, 심박수 모니터링 기능이 탑재되어 건강 관리를 한층 더 체계적으로 도와줍니다.블루투스를 통한 전화 연결 기능이 있어, 언제 어디서나 손쉽게 통화가 가능하며 중요한 연락을 놓치지 않습니다.내장된 GPS 위치 추적 시스템으로 운동 시 이동 경로를 정확하게 기록하며, 다양한 환경에서의 활동을 지원합니다.방수 기능이 추가되어 수영이나 비 오는 날에도 걱정 없이 사용할 수 있습니다.다기능 스포츠 모드가 포함되어 있어, 사용자의 다양한 운동 요구에 맞춰 최적의 운동 데이터를 제공합니다.### Instruction: Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be korean and relevant to the raw source.
### Raw Source: 창의적인 다기능 책상 정리함 사무실 학생 문구함 가정용 미니 서랍형 화장품 정리함
### Response: 창의적인 디자인의 다기능 책상 정리함은 사무실, 학교 등의 공간에서 필수 미니 서랍으로 적합합니다.
넉넉한 크기와 활동적인 설계가 결합되어, 미니 서랍으로 보다 효율적이고 시간을 절약할 수 있습니다.
문구, 화장품, 주방 도구 등 다양한 사용자의 필요에 맞춰 크기를 조절할 수 있어 공간 활용이 가능합니다.
좋은 소재 선택으로 내구성이 뛰어나 오


저장한 모델 불러와보기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
my_model_path = '/content/drive/MyDrive/ColabNotebooks/results/solar-ko-3epoch'
my_model = AutoModelForCausalLM.from_pretrained(my_model_path)
my_tokenizer = AutoTokenizer.from_pretrained(my_model_path)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

In [ ]:
 import re

def extract_response(response):
    # Define regex patterns for each type with newline or '###' as the stopping signal
    patterns = [
        (r"### Product Title:\s*(.*?)(?=\n|###|$)", "Product Title"),  # Capture until next newline or ###
        (r"### Related Keywords:\s*(.*?)(?=\n|###|$)", "Related Keywords"),  # Include commas and brackets
        (r"### Detailed Description:\s*(.*?)(?=\n|###|$)", "Detailed Description"),  # Capture until next newline or ###
        (r"Product title in Korean:\s*(.*?)(?=\n|###|$)", "Product Title"),
        (r"Product Title:\s*(.*?)(?=\n|###|$)", "Product Title"),
        (r"Detailed Description\s*:\s*(.*?)(?=\n|###|$)", "Detailed Description"),
        (r"Related keywords:\s*(.*?)(?=\n|###|$)", "Related Keywords"),
        (r"\[INST\].*?\[/INST\]\s*(.*?)(?=\n|###|$)", "General"),
        (r"<s>\[INST\].*?\[/INST\]\s*(.*?)(?=\n|###|$)", "General")
    ]

    for pattern, type_ in patterns:
        match = re.search(pattern, response, re.DOTALL)
        if match:
            title = match.group(1).strip()

            if type_ == "Related Keywords":
                # Keep brackets and commas for Related Keywords
                title = re.sub(r"[^\[\],가-힣\s\-\d]", "", title).strip()
            else:
                # Allow commas and clean up the text for other types
                title = re.sub(r"<\/?s>|<\/?INST>|raw source:.*", "", title).strip()
                title = re.sub(r"[^가-힣\s\-\d,]", "", title).strip()

            if re.search(r"[가-힣]", title):
                return title

    return "No text found."



In [ ]:
base_test_path = '/content/drive/MyDrive/ColabNotebooks/data/base0619_test.csv'

In [ ]:
test_prompt = "### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.\n\n### Raw Source:눈길 자전거 초광폭 타이어 변속 4.0 눈길용 자전거 성인 학생용 자전거 남녀공용 마운틴바이크\n### Product Title:"
test_prompt2 = "### Instruction: Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable.\
Raw Source:지능형 감지 쓰레기통 가정용 거실 주방 전동 자동 감지 방취 쓰레기통 대용량 친환경 쓰레기통\
Related Keywords:[자동 쓰레기통, 지능형 쓰레기통, 주방용 쓰레기통, 거실용 쓰레기통, 대용량 쓰레기통, 방취 쓰레기통, 친환경 쓰레기통]\
These related keywords are relevant to the raw source for the intelligent sensing trash can that automatically detects and closes when full, making it convenient for users to use in the kitchen or living room. The keyword 대용량 쓰레기통 highlights the product's large capacity, which is suitable for a family of four or more. 방취 쓰레기통 indicates that the product has a built-in odor-resistant feature, which is especially useful in the kitchen or living room where odors can be problematic.\
These keywords will help users find the product more easily when searching online, and will also"
pipe = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=200)
result = pipe(f"{test_prompt2}")
print(result[0]['generated_text'])
print(extract_response(result[0]['generated_text']))

ValueError: Input length of input_ids is 200, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


base_test = pd.read_csv(base_test_path)


pipe_product_title = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)
pipe_detailed_description = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=1024)
pipe_search_keywords = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)

results = []

for index, row in base_test.iterrows():
    #if index >= 4:
      #break
    prompt = row['text']
    result = None

    if row['type'] == "Product Title":
        result = pipe_product_title(f"{prompt}")
    elif row['type'] == "Detailed Description":
        result = pipe_detailed_description(f"{prompt}")
    elif row['type'] == "Related Keywords":
        result = pipe_search_keywords(f"{prompt}")

    if result:
        generated_text = result[0]['generated_text']
        ext = extract_response(generated_text)

        print(f"Prompt: {prompt}")
        print(f"Generated Text: {generated_text}")
        print("\n"+ext)
        print("-" * 80)

        results.append({
            'prompt': prompt,
            'type': row['type'],
            'generated_text': ext,
            'total': generated_text
        })



results_df = pd.DataFrame(results)



In [ ]:
results_df.to_csv('/content/drive/MyDrive/ColabNotebooks/generated_text/solar-ko-base-3epoch-results.csv', index=False)
results_df.to_json('/content/drive/MyDrive/ColabNotebooks/generated_text/solar-ko-base-3epoch-results.json', orient='records', lines=True)